![](https://afpglobal.org/sites/default/files/inline-images/Logo-ITESO-Principal%203-11-2020.png)


<font color= #555555> <font size = 3> Abstract </font>
    
><font color= #6B6B6B> <font size = 2> Este documento fue elaborado por tu Ana Cristina Cruz Delgadillo, como un trabajo parcial para la materia de Microestructura y Sistemas de Trading - ITE1731, la cual es parte del curriculum de la licenciatura en Ingeniería Financiera, ofertada por la universidad ITESO. En el presente trabajo se plantea la respuesta a la siguiente pregutna: ¿Qué estrategia de inversión propondrías si tu trabajo fuera administrar 1 Millón de pesos?</font>


<font color= #6B6B6B> <font size = 3> 0. Librerías y Dependencias </font>

>Para correr este notebook es necesario tener instaladas y/o en el archivo requirements.txt las siguientes librerías :

    
    
>pandas>=1.1.1
    
>numpy>=1.19.1
    
>jupyter>=1.0.0
    
>yfinance>=0.1.54

<font color= #6B6B6B> <font size = 3> 1. Introducción </font>

>Existen dos maneras de invertir en el mercado de valores: la inversión pasiva y la inversión pasiva y la inversión activa. La inversión pasiva se refiere a cualquier portafolio que simule el comportamiento de un índice, es decir, que se compre aquellos activos que forman parte del índice y se mantenga la posición respetando esa ponderación a través del tiempo. Por otro lado, está la inversión activa, en la que se pretende hacer modificaciones para obtener mayores rendimientos.
    

    
>Ambas estrategias tienen sus ventajas y desventajas. Éstas dependen del mercado en el que se estén ejecutando las compras, pues no es lo mismo un mercado de un país en desarrollo a un mercado de un país desarrollado, en el que los movimientos en el mercado son mayores.


<font color= #6B6B6B> <font size = 3> 2. Objetivos </font>

><font color= #6B6B6B> El objetivo de este laboratorio es evaluar ambas posibilidades de inversión en el mercado mexicano y concluir que en un país en desarrollo existen diferencias significativas entre ellas.


<font color= #6B6B6B> <font size = 3> 3. Datos </font>

><font color= #6B6B6B>Primero, se obtuvo el nombre de todos los archivos de la carpeta para así importarlos todos en una función y poder descargar los precios de interés.

In [6]:
# Importar las librerias necesarias para la ejecución del laboratorio 1
import yfinance as yf
from os import listdir, path
from os.path import isfile, join
import numpy as np
from datetime import timedelta, datetime
from functions import get_data
import pandas as pd

In [3]:
# obtener los nombres de los archivos
abspath = path.abspath('files/NAFTRAC_holdings')
all_files = [f[:-4] for f in listdir(abspath) if isfile(join(abspath, f))]
all_files = sorted(all_files,key= lambda string: datetime.strptime(string[8:], '%d%m%y') )

In [4]:
# Función para leer todos los archivos del NAFTRAC
def get_data(all_files):
    data_archivos = {}
    for i in all_files:
        data = pd.read_csv('files/NAFTRAC_holdings/' + i + '.csv', skiprows=2, header=None)
        data.columns = list(data.iloc[0, :])
        data = data.iloc[:, pd.notnull(data.columns)]
        data = data.iloc[1:-1].reset_index(inplace=False, drop=True)
        data['Precio'] = [i.replace(',', '') for i in data['Precio']]
        data['Ticker'] = [i.replace('*', '') for i in data['Ticker']]
        convert_dict = {'Ticker': str, 'Nombre': str, 'Peso (%)': float, 'Precio': float}
        data = data.astype(convert_dict)
        data['Peso (%)'] = data['Peso (%)'] / 100
        data_archivos[i] = data
    return data_archivos

In [8]:
data_archivos = get_data(all_files)
i_fechas = [j.strftime('%Y-%m-%d') for j in sorted([pd.to_datetime(i[8:]).date() for i in all_files])]

In [9]:
# Descargar y acomodar datos
tickers = []
for i in all_files:
    l_tickers = list(data_archivos[i]['Ticker'])
    [tickers.append(i + '.MX') for i in l_tickers]
global_tickers = np.unique(tickers).tolist()

global_tickers = [i.replace('GFREGIOO.MX', 'RA.MX') for i in global_tickers]
global_tickers = [i.replace('MEXCHEM.MX', 'ORBIA.MX') for i in global_tickers]
global_tickers = [i.replace('LIVEPOLC.1.MX', 'LIVEPOLC-1.MX') for i in global_tickers]

[global_tickers.remove(i) for i in ['MXN.MX', 'USD.MX', 'KOFL.MX','KOFUBL.MX' ,'BSMXB.MX']]


data = yf.download(global_tickers, start="2017-12-30", end="2020-09-01", actions=False,
                   group_by="close", interval='1d', auto_adj=False, prepost=False, threads=False)

[*********************100%***********************]  39 of 39 completed


In [10]:
# convertir columna de fechas
data_close = pd.DataFrame({i: data[i]['Close'] for i in global_tickers})

# tomar solo las fechas de interes (utilizando conjuntos)
ic_fechas = sorted(list(set(data_close.index.astype(str).tolist()) & set(i_fechas)))
# localizar todos los precios
precios = data_close.iloc[[int(np.where(data_close.index == i)[0]) for i in i_fechas]]

><font color= #6B6B6B>Como resultado tenemos un Dataframe con los precios que se vana a utilizar para hacer la simulación de la inversión. A continuación se muestran los precios resultantes descargados de Yahoo Finance.

In [11]:
precios

,AC.MX,ALFAA.MX,ALPEKA.MX,ALSEA.MX,AMXL.MX,ASURB.MX,BBAJIOO.MX,BIMBOA.MX,BOLSAA.MX,CEMEXCPO.MX,...,ORBIA.MX,NEMAKA.MX,OMAB.MX,PE&OLES.MX,PINFRA.MX,Q.MX,TLEVISACPO.MX,VESTA.MX,VOLARA.MX,WALMEX.MX
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-31,135.050003,23.360001,26.860001,60.880001,17.440001,362.220001,39.330002,45.400002,35.619999,15.480,...,52.750000,15.540,94.989998,431.350006,190.779999,46.500000,77.019997,25.639999,15.930000,46.639999
2018-02-28,130.839996,22.389999,24.420000,64.260002,17.340000,334.029999,41.049999,44.110001,34.939999,12.450,...,52.840000,13.410,92.110001,411.940002,192.380005,49.580002,64.510002,25.900000,16.780001,44.000000
2018-03-28,125.709999,23.290001,25.260000,63.529999,17.250000,306.420013,38.910000,39.779999,34.740002,12.030,...,55.639999,14.940,88.650002,367.260010,180.740005,50.320000,57.830002,26.990000,14.650000,46.259998
2018-04-30,129.259995,24.010000,26.809999,69.559998,17.309999,336.329987,39.889999,43.529999,36.080002,11.700,...,58.419998,14.110,99.040001,392.540009,191.639999,52.209999,67.120003,27.000000,11.690000,51.970001
2018-05-31,119.680000,20.480000,29.750000,65.410004,15.500000,316.660004,37.799999,38.060001,32.509998,11.860,...,56.189999,13.500,95.379997,336.880005,179.300003,51.049999,66.599998,25.070000,10.300000,50.060001
2018-06-29,122.260002,23.100000,29.549999,68.400002,16.570000,315.369995,41.340000,38.689999,33.320000,13.060,...,57.430000,13.610,103.570000,356.040009,177.440002,49.110001,75.379997,25.920000,10.100000,52.360001
2018-07-31,123.720001,25.360001,31.540001,64.290001,15.970000,331.589996,44.029999,39.660000,35.529999,13.890,...,65.250000,15.360,113.370003,316.010010,193.050003,49.910000,74.190002,29.530001,13.680000,54.430000
2018-08-31,117.559998,25.129999,28.190001,68.059998,16.049999,357.920013,44.500000,39.750000,38.610001,13.530,...,64.690002,15.960,126.019997,324.739990,202.440002,47.889999,68.940002,27.129999,13.960000,52.889999
2018-09-28,120.879997,24.129999,30.520000,63.669998,15.040000,382.630005,46.180000,39.910000,38.259998,13.140,...,64.360001,14.040,133.449997,322.369995,199.089996,49.880001,66.540001,28.850000,14.060000,57.029999




<font color= #6B6B6B> <font size = 3> 4. Procesos </font>
    
><font color= #6B6B6B> <font size = 2> Inversion pasiva </font>

><font color= #6B6B6B>Para comenzar la estrategia de inversión pasiva, comenzamos con el supuesto de que se tiene $1'000,000.00 para invertir y las comisiones son de 0.125%.

In [13]:
df_pasiva = pd.DataFrame(index= np.arange(0,len(all_files)+1),columns=['timestamp','capital', 'rend', 'rend_acum'])
k = 1000000
# comisiones
c = 0.00125

In [14]:
df_pasiva.capital[0]=k
df_pasiva.timestamp[0] = '2018-01-30'
df_pasiva.timestamp[1:] = i_fechas
m=0
pos_datos = data_archivos[all_files[0]][['Ticker', 'Nombre', 'Peso (%)']]
pos_datos=pd.DataFrame(pos_datos)
c_activos = ['KOFL', 'KOFUBL', 'BSMXB', 'MXN', 'USD']            #tickers a eliminar
pos_datos.drop(list(pos_datos[list(pos_datos['Ticker'].isin(c_activos))].index), inplace=True)  # eliminar tickers
pos_datos['Ticker'] = pos_datos['Ticker'] + '.MX'
pos_datos['Ticker'] = pos_datos['Ticker'].replace(['LIVEPOLC.1.MX','MEXCHEM.MX','GFREGIOO.MX'],
                                                      ['LIVEPOLC-1.MX','ORBIA.MX','RA.MX'])

><font color= #6B6B6B>Se hace la compra en enero de 2018 y se dejan las posiciones hasta agosto 2020. Por lo tanto, las comisiones sólo se cobran en la compra de enero 2018 y vemos como va evolucionando el capital.

In [15]:
pos_datos['Precio'] = [precios.iloc[0, precios.columns.to_list().index(j)] for j in pos_datos['Ticker']]
pos_datos['Capital'] = pos_datos['Peso (%)'] * k - pos_datos['Peso (%)'] * k * c
pos_datos['Titulos'] = np.round(pos_datos['Capital'] / pos_datos['Precio'])
pos_datos['Postura'] = pos_datos['Titulos']*pos_datos['Precio']
pos_datos['Comision'] = pos_datos['Postura'] * c
pos_cash = k - pos_datos['Postura'].sum() - pos_datos['Comision'].sum()
df_pasiva.capital[1] = pos_datos['Postura'].sum() + pos_cash

In [16]:
for i in range(1,len(all_files)):
    precio= [precios.iloc[i, precios.columns.to_list().index(j)] for j in pos_datos['Ticker']]
    postura = pos_datos['Titulos']*precio
    df_pasiva.capital[i+1] = postura.sum()+pos_cash

df_pasiva.rend = df_pasiva.capital/df_pasiva.capital.shift(1)-1
df_pasiva.rend_acum = np.cumsum(df_pasiva.rend)

><font color= #6B6B6B>Como resultado tenemos la siguiente table en la que observamos los rendimientos del portafolio de inversión pasiva. Se muestra a continuación:

In [17]:
df_pasiva

,timestamp,capital,rend,rend_acum
0,2018-01-30,1000000,NaN,NaN
1,2018-01-31,998804,-0.00119572,-0.00119572
2,2018-02-28,943295,-0.0555759,-0.0567716
3,2018-03-28,918852,-0.025912,-0.0826836
4,2018-04-30,961746,0.0466814,-0.0360022
5,2018-05-31,890422,-0.0741601,-0.110162
6,2018-06-29,949821,0.0667081,-0.0434541
7,2018-07-31,985774,0.0378525,-0.00560163
8,2018-08-31,984428,-0.00136549,-0.00696712
9,2018-09-28,983487,-0.000955717,-0.00792284


<font color= #6B6B6B> <font size = 2> Inversion activa </font>

><font color= #6B6B6B> Para la inversión activa se considera el portafolio inicial pasado y se identifica el activo con mayor participación, de este activo se reduce la participación a la mitad y lo sobrante se agrega a cash. Con el nuevo monto en cash se comprará el mismo activo siempre que el activo en el día tenga un comportamiento bajista, es decir, que su precio de apertura sea mayor al precio de cierre. Lo anterior mencionado con una periodicidad diaria.

In [20]:
#identificar el activo con mayor participación:

mayor = pos_datos[pos_datos['Peso (%)'] == max(pos_datos['Peso (%)'])]['Ticker'][0] # devuelve el ticker

In [23]:
#Obtener datos de apertura
data_open = pd.DataFrame({i: data[i]['Open'] for i in global_tickers})
df_activa = pd.DataFrame(index= np.arange(0,len(all_files)+1),columns=['timestamp','capital', 'rend', 'rend_acum'])
k = 1000000
c = 0.00125
df_activa.capital[0]=k
df_activa.timestamp[0] = '2018-01-30'
df_activa.timestamp[1:] = i_fechas
df_operaciones = pd.DataFrame(index=np.arange(len(data_close)), columns=['timestamp', 'titulos_totales', 'titulos_compra',
                              'precio', 'comision','comision_acum'])
act_datos = data_archivos[all_files[0]][['Ticker', 'Nombre', 'Peso (%)']]
act_datos=pd.DataFrame(act_datos)
c_activos = ['KOFL', 'KOFUBL', 'BSMXB', 'MXN', 'USD']            #tickers a eliminar
act_datos.drop(list(act_datos[list(act_datos['Ticker'].isin(c_activos))].index), inplace=True)  # eliminar tickers
act_datos['Ticker'] = act_datos['Ticker'] + '.MX'
act_datos['Ticker'] = act_datos['Ticker'].replace(['LIVEPOLC.1.MX','MEXCHEM.MX','GFREGIOO.MX'],
                                                      ['LIVEPOLC-1.MX','ORBIA.MX','RA.MX'])
act_datos['Peso (%)'] = act_datos['Peso (%)'].replace(act_datos['Peso (%)'][act_datos.Ticker==mayor][0],
                                                          act_datos['Peso (%)'][act_datos.Ticker==mayor][0]/2)

In [24]:
act_datos['Precio'] = [precios.iloc[0, precios.columns.to_list().index(j)] for j in act_datos['Ticker']]
act_datos['Capital'] = act_datos['Peso (%)'] * k - act_datos['Peso (%)'] * k * c
act_datos['Titulos'] = np.round(act_datos['Capital'] / act_datos['Precio'])
act_datos['Postura'] = act_datos['Titulos']*act_datos['Precio']
act_datos['Comision'] = act_datos['Postura'] * c

act_cash = k - act_datos['Postura'].sum() - act_datos['Comision'].sum()
df_activa.capital[1] = act_datos['Postura'].sum() + act_cash

In [25]:
for i in range(len(data_close)-1):
    if data_open[mayor][i]>data_close[mayor][i]:
        df_operaciones.timestamp[i] = data_close.index[i]
        df_operaciones.titulos_compra[i] = np.round((act_cash*0.1)/data_close[mayor][i+1])
        df_operaciones.precio[i]=data_close[mayor][i+1]
        df_operaciones.comision[i] = data_close[mayor][i+1] + df_operaciones.titulos_compra[i]*c
        act_cash = act_cash- df_operaciones.titulos_compra[i]*df_operaciones.precio[i]-df_operaciones.comision[i]
    else:
        pass

><font color= #6B6B6B> Una vez obtenidos los totales de las compras de las operaciones se agregan al Dataframe de inversión pasiva para ser completada. 


<font color= #6B6B6B> <font size = 3> 5. Conclusiones </font>

><font color= #6B6B6B> La ventaja de la inversión pasiva es el bajo costo de transacción, mientras que en la activa los costos de transacción son más elevados. Quizá no sea conveniente realizar la inversión activa porque los movimientos en el mercado mexicano no son lo suficientemente grandes para compensar el costo de transacción.